# Training

This notebook serves as a walkthrough for training with trapper package.

In [ ]:
cd ..

In [ ]:
# Setting up the correct path

from copy import deepcopy
import os
import json
from typing import Any, Dict, List, Tuple, Union
import warnings

from jury import Jury
import requests
from tqdm import tqdm

from trapper.training.train import run_experiment

Set logging configuration.

In [ ]:
import logging
import sys

# You can customize your logger below.
logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)

In [ ]:
# Define constants
EXPERIMENT_NAME = "roberta-base-training-example"

WORKING_DIR = os.getcwd()
EXPERIMENTS_DIR = os.path.join(WORKING_DIR, "examples/experiments")

## Helper functions

Some useful helper functions to ease training.

In [ ]:
def get_dir_from_task(path: str, task: str):
    task = "unnamed-task" if task is None else task
    return path.format(task=task)

def start_experiment(config: str, task: str, ext_vars: Dict[str, str]):
    result = run_experiment(
        config_path=config,
        ext_vars=ext_vars,
    )

    print("Training complete.")
    return result

In [ ]:
TASK = "question-answering"
TASK_DIR = get_dir_from_task(os.path.join(EXPERIMENTS_DIR, "{task}"), task=TASK)
DATASET_DIR = os.path.join(TASK_DIR, "datasets")
EXPERIMENT_DIR = os.path.join(TASK_DIR, EXPERIMENT_NAME)
MODEL_DIR = os.path.join(EXPERIMENT_DIR, "model")
CHECKPOINT_DIR = os.path.join(EXPERIMENT_DIR, "checkpoints")
OUTPUT_DIR = os.path.join(EXPERIMENT_DIR, "outputs")

In [ ]:
ext_vars = {
    # Used to feed the jsonnet config file with file paths
    "OUTPUT_PATH": OUTPUT_DIR,
    "CHECKPOINT_PATH": CHECKPOINT_DIR
}

CONFIG_PATH = os.path.join(TASK_DIR, "experiment.jsonnet")  # default experiment params

In [ ]:
result = start_experiment(
    config=CONFIG_PATH,
    task=TASK,
    ext_vars=ext_vars,
)

In [ ]:
result

# Inference

In this section, usage of pipeline for inference is illustrated.

In [ ]:
from trapper.pipelines.question_answering_pipeline import SquadQuestionAnsweringPipeline
from trapper.pipelines.pipeline import create_pipeline_from_checkpoint

## Helper Functions

Some helper functions for inference steps.

In [ ]:
def save_json(samples: List[Dict], path: str):
    with open(path, "w") as jf:
        json.dump(samples, jf)


def load_json(path: str):
    with open(path, "r") as jf:
        return json.load(jf)


def prepare_samples(data: Union[str, Dict]):
    if isinstance(data, str):
        data = load_json(data)
    data = data["data"]
    qa_samples = []

    for article in data:
        for paragraph in article["paragraphs"]:
            for qa in paragraph["qas"]:
                sample = {}
                sample["context"] = paragraph["context"]
                sample["question"] = qa["question"]
                sample["gold_answers"] = [ans["text"] for ans in qa["answers"]]
                qa_samples.append(sample)

    return qa_samples


def prepare_samples_for_pipeline(samples: List[Dict]):
    pipeline_samples = deepcopy(samples)
    for sample in pipeline_samples:
        sample.pop("gold_answers")
    return pipeline_samples


def predict(pipeline, samples, **kwargs):
    pipeline_samples = prepare_samples_for_pipeline(samples)
    predictions = pipeline(pipeline_samples, **kwargs)
    for i, prediction in enumerate(predictions):
        samples[i]["predicted_answer"] = prediction[0]["answer"].text
    return samples

In [ ]:
SQUAD_DEV = "/home/devrimcavusoglu/lab/bb/nqg/datasets/squad/squad-qa/dev-v1.1.json"
EXPORT_PATH = "/home/devrimcavusoglu/lab/bb/nqg/notebooks/qa-predictions_pipeline.json"

PRETRAINED_MODEL_PATH = "/home/devrimcavusoglu/lab/bb/nqg/models/question_answering/roberta-large_ep_5_ebs_384_lr_5e-05-v1"
EXPERIMENT_CONFIG = os.path.join(PRETRAINED_MODEL_PATH, "experiment_config.json")

In [ ]:
qa_pipeline = create_pipeline_from_checkpoint(
    checkpoint_path=PRETRAINED_MODEL_PATH,
    experiment_config_path=EXPERIMENT_CONFIG,
    task="squad-question-answering",
    device=0
)

In [ ]:
samples = prepare_samples(SQUAD_DEV)

In [ ]:
predictions = predict(qa_pipeline, samples)

In [ ]:
save_json(predictions, EXPORT_PATH)

In [ ]:
references = [sample["gold_answers"] for sample in predictions]
hypotheses = [sample["predicted_answer"] for sample in predictions]

In [ ]:
jury = Jury(metrics=["squad_f1", "squad_em"])

In [ ]:
jury.evaluate(references=references, predictions=hypotheses)